In [ ]:
import numpy as np
import torch
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# from utilities import *
import numpy.random as random

torch.manual_seed(0)
np.random.seed(0)

This notebook contains code to visualize PFNN predictions on the Kolmogorov Flow and compare them to baseline models. Additionally, the notebook can easily be adapted to visualizing predictions on other 2D Navier-Stokes equation systems.

In the mean time, we visualize and compare the learned invariant statistics of long term predictions to the true invariant measure of the system.

## Preparation

In [ ]:
import sys
sys.path.append('./model')
from utils.rnn_base import *
from utils.vae_base import *
from pfnn import *


# load models
device = torch.device('cuda:0' if torch.cuda.is_available() else 'mps')
model_pfnn = torch.load('your_model_path', map_location=device)
model_mno = torch.load('your_model_path', map_location='cpu')
model_lstm = torch.load('your_model_path', map_location=device)
model_koopman = torch.load('your_model_path', map_location=device)

## short-mid term prediction

## Invariant Statistics

### Turbulent kinetic energy (TKE)

### Mean state